In [1]:
import pandas as pd


# Read the TSV file into a Pandas DataFrame
df1 = pd.read_csv('../data/raw/filtered.tsv', sep='\t', index_col=0)

# Display the DataFrame
df1.head(10)


,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348
5,I'm not gonna have a child... ...with the same...,I'm not going to breed kids with a genetic dis...,0.703185,0.206522,0.950956,0.035846
6,"They're all laughing at us, so we'll kick your...",they're laughing at us. We'll show you.,0.618866,0.230769,0.999492,0.000131
7,Maine was very short on black people back then.,there wasn't much black in Maine then.,0.720482,0.187500,0.963680,0.148710
8,"Briggs, what the hell's happening?","Briggs, what the hell is going on?",0.920373,0.000000,0.159096,0.841071
9,"Another one simply had no clue what to do, so ...","another simply didn't know what to do, so when...",0.877540,0.101695,0.055371,0.930472


In [2]:
str1, str2 = [], []
for i in range(50):
    str1.append(df1.iloc[i]['reference'])
    str2.append(df1.iloc[i]['translation'])


In [3]:
df2 = pd.read_csv('../data/external/paradetox.tsv', sep='\t', index_col=0)

# Display the DataFrame
df2.head(5)

,neutral1,neutral2,neutral3
toxic,,,
he had steel balls too !,he was brave too!,NaN,NaN
"dude should have been taken to api , he would be right at home with all the other knuckleheads there",It would have been good if he went to api. He ...,NaN,NaN
"im not gonna sell the fucking picture , i just want to contribute to the fucking article .","I'm not gonna sell the picture, i just want to...",NaN,NaN
the garbage that is being created by cnn and other news agencies is outrageous .,the news that is being created by cnn and othe...,The news that is being created by cnn and othe...,the garbage that is being created by cnn and o...
the reason they dont exist is because neither is a pathological liar like trump .,The reason they don't exist is because neither...,NaN,NaN


In [4]:
import nltk
from nltk.corpus import wordnet
from nltk.wsd import lesk

In [5]:
word = 'dude'
context_sentence = "dude should have been taken to api , he would be right at home with all the other knuckleheads there"


In [6]:
def get_synonym_by_context(word, context_sentence):
    # Disambiguate the word in the context sentence
    sense = lesk(context_sentence, word)

    # Find synonyms for the disambiguated sense
    synonyms = []
    if sense:
        for lemma in sense.lemmas():
            synonyms.append(lemma.name())

    # Remove duplicates and display the closest synonym
    synonyms = list(set(synonyms))
    return synonyms


In [7]:
print("Closest synonym for", word, "in the context:", get_synonym_by_context(word, context_sentence))

Closest synonym for dude in the context: ['fellow', 'buster', 'dude']


In [8]:
df3 = pd.read_csv('../data/external/jigsaw-toxic-comment-classification-challenge/train.csv')
df3.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [9]:
df3['comment_text'].iloc[6], df3['toxic'].iloc[6]

('COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK', 1)

In [10]:
df3['comment_text'].iloc[0], df3['toxic'].iloc[0]

("Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
 0)

In [11]:
df3.loc[df3['toxic'] == 1]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
...,...,...,...,...,...,...,...,...
159494,fef4cf7ba0012866,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1
159514,ff39a2895fc3b40e,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0
159541,ffa33d3122b599d6,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0
159546,ffb47123b2d82762,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0


In [12]:
df3.loc[df3['toxic'] == 0]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [13]:
import sys
sys.path.insert(0, '../src/metric')

In [14]:
sys.executable

'/usr/local/bin/python3'

In [15]:
from metric import calculate_metric

INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX


In [16]:
calculate_metric(str1, str2)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443


Calculating style of predictions


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /SkolkovoInstitute/roberta_toxicity_classifier/resolve/main/vocab.json HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /s-nlp/roberta_toxicity_classifier/resolve/main/vocab.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /SkolkovoInstitute/roberta_toxicity_classifier/resolve/main/config.json HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /s-nlp/roberta_toxicity_classifier/resolve/main/config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

Calculating BLEU similarity
Calculating similarity by Wieting subword-embedding SIM model


100%|██████████| 2/2 [00:00<00:00, 239.06it/s]
INFO:fairseq.file_utils:loading archive file ../src/metric/models/cola


Calculating CoLA acceptability stats
../src/metric/models/cola/cola-bin


TypeError: 'NoneType' object is not iterable